In [1]:
import pandas as pd

In [3]:
def get_data(count = 100000):
    rt = []    
    counter = 0
    with open('../Data/usa_00016.dat') as f:
        for line in f:
            rt.append(line.strip())
            counter += 1
            if counter >= count:
                return rt
    return rt

data = get_data()

In [4]:
def get_positions():

    rt = {'year' : [0, 4],
                 'gender' : [52, 53],
                 'age' : [53, 56],
                 'race' : [57, 57],
                 'income' : [60, 67]
                }
        
    return rt

In [5]:
def get_data_frame(data):
    rt = []
    positions = get_positions()
    for row in data:
        new_row = []
        for key in positions:
            start = positions[key][0]
            end = positions[key][1]
            if start == end:
                f = row[start]
            else:
                f = row[start:end]
            new_row.append(f)
        rt.append(new_row)
    df = pd.DataFrame(rt, columns = ['YEAR', 'SEX', 'AGE', 'RACE', 'INCOME'])
    return df
        


In [6]:
data = get_data()  
df = get_data_frame(data)

In [7]:
df.head()

,YEAR,SEX,AGE,RACE,INCOME
0,2009,1,051,1,0003800
1,2009,2,064,1,0036800
2,2009,1,068,1,0054600
3,2009,2,061,2,0006000
4,2009,1,038,2,0014000


## Read the codebook

In [8]:
import json

In [9]:
def read_json(f_name):
    with open(f_name) as j:  
        codebook = json.load(j)
    return codebook

codes = read_json('../Data/codebook.json')

## Build a transformer

In [11]:
import multiprocessing
from multiprocessing import Pool

In [17]:
header = list(df)

In [13]:
rows = df.as_matrix()

In [114]:
# It works well with variables defined within the kernel
# It fails when we call other functions inside the process
# But it won't error out, it just returns a None Type 

def transform_row(row):
    
    new_row = row[:]
    
    for idx, each in enumerate(row):
        
        h = header[idx]

        if h == 'YEAR':
            new_row[idx] = each
            continue

        key = str(int(each))

        # start the special cases here 
        if h == 'INCOME' or h == 'RACE':
            updated = int(key)
        else:
            updated = codes[h][key]
        new_row[idx] = updated

    return new_row  

In [115]:
data = get_data()  
df = get_data_frame(data)
rows = df.as_matrix() 

In [123]:
def magic_loop(rows):
    pool = Pool(processes=multiprocessing.cpu_count())
    transformed_rows = pool.map(transform_row, rows)
    pool.close() 
    pool.join()
    df = pd.DataFrame(transformed_rows, columns = header)
    return df

In [124]:
df = magic_loop(rows)

In [125]:
df.head()

,YEAR,SEX,AGE,RACE,INCOME
0,2009,Male,51,1,3800
1,2009,Female,64,1,36800
2,2009,Male,68,1,54600
3,2009,Female,61,2,6000
4,2009,Male,38,2,14000
